# 对话式 RAG - Conversational RAG
在许多问答应用程序中，我们希望允许用户进行来回对话，这意味着应用程序需要某种过去问题和答案的“记忆”，以及将这些问题和答案纳入当前思维的逻辑。

在本指南中，我们重点介绍如何添加逻辑来整合历史消息。此处介绍了有关聊天记录管理的更多详细信息。

我们将介绍两种方法：

 - 链 Chains，我们始终在其中执行检索步骤；

 - 代理 Agents，我们让 LLM 自行决定是否以及如何执行检索步骤（或多个步骤）。
   

In [1]:
import os
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
)

In [3]:
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

In [4]:
# 安装zhipu
# pip install zhipuai

In [5]:
# from langchain_community.embeddings import ZhipuAIEmbeddings
# # import langchain_community
# # langchain_community.embeddings.__all__

# embeddings = ZhipuAIEmbeddings()

In [6]:
import bs4

from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = WebBaseLoader(
    web_paths=("https://ouzhoubei.co/article-794-1.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(id=("article_content"))
    )
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vectorstore.as_retriever()

system_prompt = (
    "您是问答任务的助手。"
    "使用以下检索到的上下文来回答问题。如果您不知道答案，请说您不知道。最多使用三句话并保持答案简洁。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
response = rag_chain.invoke({"input": "意大利队有多少位球员?"})
response["answer"]

'意大利队有26位球员。'

请注意，我们使用了内置链构造函数 create_stuff_documents_chain 和 create_retrieval_chain，因此我们的解决方案的基本要素是：

 - retriever
 - prompt
 - LLM
   
这将简化合并聊天历史记录的过程。

## 添加聊天记录

我们构建的链直接使用输入查询来检索相关上下文。但在对话设置中，用户查询可能需要对话上下文才能被理解。例如，考虑以下交流：

> 人类：“什么是任务分解？”
>
> 人工智能：“任务分解涉及将复杂的任务分解为更小、更简单的步骤，以便代理或模型更易于管理。”
>
> 人类：“它常见的有哪些？”

为了回答第二个问题，我们的系统需要理解“它”指的是“任务分解”。  
我们需要更新现有应用的两件事：  

 - Prompt 提示：更新我们的提示以支持历史消息作为输入。

 - Contextualizing questions 情境化问题：添加一个子链，该子链接受最新的用户问题并在聊天历史的上下文中重新表述它。这可以简单地被认为是构建一个新的“历史感知”检索器。  
   而之前我们有：

   -  query查询 -> retriever检索器
     
      现在我们将有：
      
   - （query查询，conversation history对话历史）-> LLM大模型 -> rephrased query重新表述的查询 -> retriever检索器
  
**将问题情境化 Contextualizing the question**  

首先，我们需要定义一个子链，该子链接收历史消息和最新的用户问题，并且如果问题引用了历史信息中的任何信息，则重新表述该问题。  

我们将使用一个提示模板，其中包含一个名为 "chat_history" 的 MessagesPlaceholder 变量。  
这使我们能够使用 "chat_history" 输入键向提示传递消息列表，这些消息将在系统消息之后和包含最新问题的人类消息之前插入。 

请注意，我们利用辅助函数 create_history_aware_retriever 来完成此步骤，该函数管理 chat_history 为空的情况，否则按顺序应用 prompt | llm | StrOutputParser() | 检索器。  

create_history_aware_retriever 构造一个链，该链接受键 input 和 chat_history 作为输入，并具有与检索器相同的输出模式。

In [8]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "根据聊天记录和用户最新的问题，"
    "该问题可能涉及聊天历史中的上下文，"
    "形成一个独立的问题，"
    "这个问题是可以在不了解聊天历史的情况下被理解的。"
    "不要回答这个问题，只需在需要时进行重新表述，否则保持原样返回。"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system",contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])

history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

此链将在我们的检索器前添加输入查询的改写，从而让检索过程融入对话的上下文信息。

现在我们可以构建完整的QA问答链条了。这个过程很简单，只需将检索器更新为我们新创建的history_aware_retriever即可。

同样，我们将使用create_stuff_documents_chain来生成一个question_answer_chain，  
其输入键包括context（上下文）、chat_history（聊天历史）和input（输入）——它接收检索到的上下文，连同对话历史和查询一起，来生成答案。

我们通过create_retrieval_chain来构建最终的RAG（检索增强生成）链条。这条链条依次应用history_aware_retriever和question_answer_chain，为了方便起见，会保留诸如检索到的上下文等中间输出。  
该链条的输入键包括input（输入）和chat_history（聊天历史），其输出则包含input（输入）、chat_history（聊天历史）、context（上下文）和answer（答案）。

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])

question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

让我们尝试一下。下面我们将提出一个问题以及一个需要结合上下文才能给出合理回答的跟进问题。  
由于我们的链条包含了一个"chat_history"（聊天历史）输入，调用者需要管理这个聊天历史。  
我们可以通过将输入和输出的消息追加到一个列表中来实现这一点：

In [10]:
from langchain_core.messages import AIMessage,HumanMessage

chat_history = []

question = "2024年，意大利队有多少位球员?"
ai_msg_1 = rag_chain.invoke({
    "input":question,
    "chat_history":chat_history
})
print(ai_msg_1["answer"])
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=ai_msg_1["answer"])
])

question2 = "一一列出上述这些球员信息,按照：号码-姓名-俱乐部的格式输出。"
ai_msg_2 = rag_chain.invoke({
    "input":question2,
    "chat_history":chat_history
})

print(ai_msg_2["answer"])

根据提供的信息，2024年意大利队有26位球员。
1. 18号：巴雷拉-国际米兰
2. 19号：雷特吉-热那亚
3. 20号：扎卡尼-拉齐奥
4. 21号：法乔利-尤文图斯
5. 22号：沙拉维-罗马
6. 23号：巴斯托尼-国际米兰
7. 24号：坎比亚索-尤文图斯
8. 25号：弗洛伦肖-维罗纳
9. 26号：梅雷特-那不勒斯


In [11]:
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=ai_msg_2["answer"])
])

question3 = "上述球员有几位来自尤文图斯，分别是谁"
ai_msg_3 = rag_chain.invoke({
    "input":question3,
    "chat_history":chat_history
})

print(ai_msg_3["answer"])

来自尤文图斯的球员有两位，分别是21号的法乔利和24号的坎比亚索。


**聊天记录的状态管理**  

这里我们介绍了如何添加应用程序逻辑来整合历史输出，但我们仍在手动更新聊天记录并将其插入到每个输入中。  
在真正的问答应用程序中，我们需要某种方式来保存聊天记录，以及某种方式来自动插入和更新它。

为此，我们可以使用以下两个组件：

 - **BaseChatMessageHistory**：用来存储聊天历史记录。这个类允许你保存和管理对话中的一系列消息，这对于构建有状态的对话系统至关重要，因为它可以帮助模型理解会话的上下文。

 - **RunnableWithMessageHistory**：这是一个包装器，它将一个LCEL（LangChain 表达语言）和一个BaseChatMessageHistory结合在一起，负责在每次调用输入时注入聊天历史，并在每次调用后更新历史记录。这个类简化了将聊天历史（即记忆）集成到对话逻辑中的过程，确保模型在生成响应时能考虑到之前的对话内容。

下面，我们实现第二个选项的一个简单示例，其中聊天历史记录存储在一个简单的字典中。LangChain 管理与 Redis 和其他技术的内存集成，以提供更强大的持久性。

RunnableWithMessageHistory 的实例为您管理聊天历史记录。  
它们接受带有键（默认为“session_id”）的配置，该键指定要获取并添加到输入中的对话历史记录，并将输出附加到相同的对话历史记录。  
以下是一个例子：

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [29]:
conversational_rag_chain.invoke(
    {"input":"2024年，意大利队有多少位球员?"},
    config={"configurable":{
        "session_id":"s1"
    }}
)["answer"]

Parent run 62a85673-ec23-49df-a705-6e25f6eadeb0 not found for run a6b4153e-c1cd-4f85-99ea-bc03d2cc863e. Treating as a root run.


'2024年，意大利队共有26位球员参加欧洲杯。'

In [30]:
conversational_rag_chain.invoke(
    {"input":"一一列出上述这些球员信息,按照：号码-姓名-俱乐部的格式输出。"},
    config={"configurable":{"session_id":"s1"}}
)["answer"]

Parent run 7713e991-3b3a-4352-9a89-36abcbb20097 not found for run ef257927-b986-4cb1-b7fa-9b633e13e98c. Treating as a root run.


'1号：多纳鲁马（巴黎圣日耳曼）2号：迪洛伦佐（那不勒斯）3号：托洛伊（亚特兰大）4号：斯皮纳佐拉（罗马）5号：阿切尔比（国际米兰）6号：博努奇（尤文图斯）7号：弗洛伦齐（AC米兰）8号：若日尼奥（阿森纳）9号：贝洛蒂（罗马）10号：因西涅（那不勒斯）11号：因莫比莱（拉齐奥）12号：戈里尼（热刺）13号：埃默森（切尔西）14号：克里斯坦特（罗马）15号：阿斯科尼（亚特兰大）16号：佩西纳（亚特兰大）17号：基耶萨（尤文图斯）18号：巴雷拉（国际米兰）19号：雷特吉（热那亚）20号：扎卡尼（拉齐奥）21号：法乔利（尤文图斯）22号：沙拉维（罗马）23号：巴斯托尼（国际米兰）24号：坎比亚索（尤文图斯）25号：弗洛伦肖（维罗纳）26号：梅雷特（那不勒斯）'

In [31]:
conversational_rag_chain.invoke(
    {"input":"上述球员有几位来自尤文图斯，分别是谁?"},
    config={"configurable":{"session_id":"s1"}}
)["answer"]

Parent run b79e22d6-c1cc-4d70-8690-0b992e2a3772 not found for run 563b8114-d97b-49a0-aaad-0d9c412e5971. Treating as a root run.


'来自尤文图斯的球员有四位，分别是：博努奇、基耶萨、巴雷拉和坎比亚索。'

可以在store中检查对话历史记录：

In [16]:
for message in store["s1"].messages:
    if isinstance(message,AIMessage):
        prefix = "AI"
    else:
        prefix = "User"
    print(f"{prefix}:{message.content} \n")

User:意大利队有多少位球员? 

AI:意大利队有26位球员。 

User:一一列出上述这些球员信息,按照：号码-姓名-俱乐部的格式输出。 

AI:18号：巴雷拉（国际米兰）19号：雷特吉（热那亚）20号：扎卡尼（拉齐奥）21号：法乔利（尤文图斯）22号：沙拉维（罗马）23号：巴斯托尼（国际米兰）24号：坎比亚索（尤文图斯）25号：弗洛伦肖（维罗纳）26号：梅雷特（那不勒斯） 

User:上述球员有几位来自尤文图斯，分别是谁? 

AI:上述球员中有两位来自尤文图斯，分别是21号法乔利和24号坎比亚索。 



为了方便起见，我们将所有必要的步骤整合到一个代码单元中：

In [17]:
import bs4
from langchain.chains import create_history_aware_retriever,create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import BaichuanTextEmbeddings

llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
    temperature=0,
)
embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

loader = WebBaseLoader(
    web_paths=("https://ouzhoubei.co/article-794-1.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(id=("article_content",))
    )
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vectorstore.as_retriever()

### 有语境的问题 ###
contextualize_q_system_prompt = (
    "根据聊天记录及用户最新的问题"
    "该问题可能涉及聊天记录中的上下文信息，重新构思一个独立的问题，"
    "使其能够在不参考聊天历史的情况下被理解。"
    "不要直接回答问题，而是在需要时对其进行重新表述，否则直接按原样返回问题。"
)

contextuallize_q_prompt = ChatPromptTemplate.from_messages([
    ("system",contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])
history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

### 回答问题 ###
system_prompt = (
    "您是问答任务的助手。"
    "使用以下检索到的上下文来回答问题。"
    "如果您不知道答案，就说您不知道。"
    "最多使用三句话，并保持答案简洁。"
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])
question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

### 状态化管理聊天记录 ###
store = {}

def get_session_hisotry(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [18]:
config = {
    "configurable":{
        "session_id":"s2"
    }
}

In [19]:
conversational_rag_chain.invoke(
    {"input":"2024年，意大利队有多少位球员?"},
    config=config
)["answer"]

Parent run 0cda3c2e-0fb6-4c4c-bf70-ccf26d915e55 not found for run c48b43de-f344-4b6a-aa0d-c644818326f2. Treating as a root run.


'在2024年的欧洲杯中，意大利队有26位球员。'

In [20]:
conversational_rag_chain.invoke(
    {"input":"一一列出上述这些球员信息,按照：号码-姓名-俱乐部的格式输出。"},
    config=config
)["answer"]

Parent run 152c17a4-ed5e-401f-a8e0-3a3ab60bcc97 not found for run a8b5b0a4-056d-4cb6-aba9-cfae56802fb0. Treating as a root run.


'1. 1号-多纳鲁马-巴黎圣日耳曼\n2. 2号-迪洛伦佐-那不勒斯\n3. 3号-迪马尔科-国际米兰\n4. 4号-布翁乔尔诺-都灵\n5. 5号-卡拉菲奥里-博洛尼亚\n6. 6号-加蒂-尤文图斯\n7. 7号-弗拉泰西-国际米兰\n8. 8号-若日尼奥-阿森纳\n9. 9号-斯卡马卡-亚特兰大\n10. 10号-佩莱格里尼-罗马\n11. 11号-拉斯帕多里-那不勒斯\n12. 12号-维卡里奥-热刺\n13. 13号-达米安-国际米兰\n14. 14号-基耶萨-尤文图斯\n15. 15号-贝拉诺瓦-都灵\n16. 16号-克里斯坦特-罗马\n17. 17号-曼奇尼-罗马\n18. 18号-巴雷拉-国际米兰\n19. 19号-雷特吉-热那亚\n20. 20号-扎卡尼-拉齐奥\n21. 21号-法乔利-尤文图斯\n22. 22号-沙拉维-罗马\n23. 23号-巴斯托尼-国际米兰\n24. 24号-坎比亚索-尤文图斯\n25. 25号-弗洛伦肖-维罗纳\n26. 26号-梅雷特-那不勒斯'

In [21]:
conversational_rag_chain.invoke(
    {"input":"有几位球员来自尤文图斯，分别是谁?"},
    config=config
)["answer"]

Parent run 639bd177-ade2-47f9-b36e-8a67f9fc0e88 not found for run 904df575-5e25-47ea-b296-2de8aaa2f3cc. Treating as a root run.


'有四位球员来自尤文图斯，分别是：6号-加蒂、14号-基耶萨、21号-法乔利和24号-坎比亚索。'

## 代理 Agents

代理（Agents）利用大型语言模型（LLMs）的推理能力在执行过程中作出决策。  
使用代理允许你将检索过程的部分自主权转移出去。  
尽管它们的行为比链条（chains）更难以预测，但在某些情况下，它们提供了一些优势：

 - 代理直接生成传递给检索器的输入，不必像上面那样明确地构建上下文关联；   
   这意味着代理能够更加灵活地根据当前情境生成查询指令，而无需人工详细定义每次检索的上下文背景。

 - 代理可以根据查询需求执行多个检索步骤，或者根据情况完全跳过检索步骤（例如，当用户发出一般性的问候时）。  
   这种灵活性使得代理能够更智能地适应不同场景，比如在需要深入信息搜索时连续执行检索，或在识别到简单互动时直接响应，无需进一步检索信息。

综上所述，代理通过动态生成检索指令和自适应调整检索策略，为交互过程提供了更高层次的动态性和智能性。

### 检索工具 Retrieval tool

代理可以访问“工具”并管理其执行。在这种情况下，我们将把检索器retriever转换为 LangChain 工具，供代理使用：

In [22]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "2024年欧洲杯各球队球衣号码汇总",
    "搜索并返回2024年欧洲杯各球队球衣号码文章的摘录。"
)
tools = [tool]

工具是指LangChain可运行组件，它们实现了通常的接口。

In [23]:
tool.invoke("一一列出2024年，意大利队球员信息,按照：号码-姓名-俱乐部的格式输出。")

'莱德红星）25号：姆拉德诺维奇（帕纳辛奈科斯）26号：比曼切维奇（布拉格斯巴达）2024年欧洲杯英格兰球员号码：1号：皮克福德（埃弗顿）2号：沃克（曼城）3号：卢克·肖（曼联）4号：赖斯（阿森纳）5号：斯通斯（曼城）6号：格伊（水晶宫）7号：萨卡（阿森纳）8号：阿诺德（利物浦）9号：凯恩（拜仁）10号：贝林厄姆（皇马）11号：福登（曼城）12号：特里皮尔（纽卡）13号：拉姆斯代尔（阿森纳）14号：孔萨（维拉）15号：邓克（布莱顿）16号：加拉格尔（切尔西）17号：伊万·托尼（布伦特福德）18号：安东尼·戈登（纽卡）19号：沃特金斯（阿斯顿维拉）20号：鲍文（西汉姆）21号：埃泽（水晶宫）22号：戈麦斯（利物浦）23号：迪恩·亨德森（水晶宫）24号：帕尔默（切尔西）25号：沃顿（水晶宫）26号：梅努（曼联）【2024年欧洲杯小组赛D组】2024年欧洲杯波兰球员号码：1号：什琴斯尼（尤文图斯）2号：萨拉蒙（波兹南莱赫）3号：达维多维茨（维罗纳）4号：瓦卢基维茨（恩波利）5号：贝德纳雷克（南安普顿）6号：皮奥特洛夫斯基（卢多戈雷茨）7号：希维德尔斯基（维罗纳）8号：莫德尔（布莱顿）9号：莱万多夫斯基（巴萨）10号：泽林斯基（那不勒斯）11号：格罗西茨基（什切青波贡）12号：斯科鲁普斯基（博洛尼亚）13号：罗曼丘克（乔治罗尼亚）14号：基维奥尔（阿森纳）15号：蒂莫特斯·普沙茨（凯泽斯劳滕）16号：布克萨（安塔利亚体育）17号：希曼斯基（雅典AEK）18号：贝雷申斯基（恩波利）19号：弗兰科夫斯基（朗斯）20号：希曼斯基（费内巴切）21号：扎莱夫斯基（罗马）22号：布尔卡（尼斯）23号：皮亚特克（伊斯坦布尔）24号：斯利兹（亚特兰大联）25号：斯科拉斯（布鲁日）26号：乌尔班斯基（博洛尼亚）2024年欧洲杯荷兰球员号码：1号：维尔布鲁根（布莱顿）2号：海特勒伊达（费耶诺德）3号：德里赫特（拜仁慕尼黑）4号：范迪克（利物浦）5号：阿克（曼城）6号：德弗里（国际米兰）7号：哈维·西蒙斯（莱比锡）8号：维纳尔杜姆（达曼协作）9号：韦霍斯特（霍芬海姆）10号：德佩（马德里竞技）11号：加克波（利物浦）12号：弗林蓬（勒沃库森）13号：拜洛（费耶诺德）14号：赖因德斯（AC米兰）15号：范德文（热刺）16号：威尔曼（埃因霍温）17号：布林德（赫罗纳）18号：马伦（多特蒙德）19号：

### 代理构造函数 Agent constructor

现在我们已经定义了工具和LLM（大型语言模型），接下来可以创建代理（Agent）了。我们将使用LangGraph来构建这个代理。  
目前，我们采用了一个高层级的接口来构造代理，但LangChain的一个优点是，这个高层级接口背后有一个低层级、高度可控的API支持。  
这意味着如果你希望修改代理的逻辑或行为，完全可以深入到更低的层次进行定制和调整。  
这样的设计既提供了便捷的快速构建能力，又保持了足够的灵活性以满足复杂的定制需求。

In [24]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm,tools)

我们现在可以尝试一下。请注意，到目前为止它还不是有状态的（我们仍然需要添加内存）

In [25]:
query = "一一列出2024年，意大利队球员信息,按照：号码-姓名-俱乐部的格式输出。"

for s in agent_executor.stream(
    {"messages":[HumanMessage(content=query)]}
):
    print(s)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'ec91016JF97oW8x', 'function': {'arguments': '{"query": "2024年意大利队欧洲杯球员名单"}', 'name': '2024年欧洲杯各球队球衣号码汇总'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 314, 'total_tokens': 348}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-11dcaa6a-6c6b-47bb-b40c-188f526b65f0-0', tool_calls=[{'name': '2024年欧洲杯各球队球衣号码汇总', 'args': {'query': '2024年意大利队欧洲杯球员名单'}, 'id': 'ec91016JF97oW8x'}], usage_metadata={'input_tokens': 314, 'output_tokens': 34, 'total_tokens': 348})]}}
---
{'tools': {'messages': [ToolMessage(content='林联合）23号：伊武希奇（帕福斯）24号：马尔科·帕萨利奇（里耶卡）25号：苏契奇（萨尔茨堡红牛）26号：巴图里纳（萨格勒布迪纳摩）2024年欧洲杯意大利球员号码：1号：多纳鲁马（巴黎圣日耳曼）2号：迪洛伦佐（那不勒斯）3号：迪马尔科（国际米兰）4号：布翁乔尔诺（都灵）5号：卡拉菲奥里（博洛尼亚）6号：加蒂（尤文图斯）7号：弗拉泰西（国际米兰）8号：若日尼奥（阿森纳）9号：斯卡马卡（亚特兰大）10号：佩莱格里尼（罗马）11号：拉斯帕多里（那不勒斯）12号：维卡里奥（热刺）13号：达米安（国际米兰）14号：基耶萨（尤文图斯）15号：贝拉诺瓦（都灵）16号：克里斯坦

LangGraph 具有内置持久性，因此我们不需要使用 ChatMessageHistory！  
相反，我们可以直接将检查点传递给我们的 LangGraph 代理

In [26]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm,tools,checkpointer=memory)

这就是我们构建对话式 RAG 代理所需的全部内容。

让我们观察一下它的行为。请注意，如果我们输入不需要检索步骤的查询，代理就不会执行该步骤：

In [27]:
config = {
    "configurable":{
        "thread_id":"t1"
    }
}

for s in agent_executor.stream(
    {"messages":[HumanMessage(content="你好，我是张三")]},
    config=config
):
    print(s)
    print("---")

{'agent': {'messages': [AIMessage(content='你好，张三！有什么我可以帮助你的吗？', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 294, 'total_tokens': 305}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9eb67dd9-03d5-4ec7-b819-7d017470781c-0', usage_metadata={'input_tokens': 294, 'output_tokens': 11, 'total_tokens': 305})]}}
---


此外，如果我们输入确实需要检索步骤的查询，代理将生成工具的输入：

In [28]:
query = "列出2024年意大利队中属于尤文图斯俱乐部的球员信息，按照：号码-姓名-俱乐部的格式输出。"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='为了提供准确的信息，我需要进行一次搜索来找到最新的关于2024年意大利国家队成员和他们所属俱乐部的数据。请稍等片刻。', additional_kwargs={'tool_calls': [{'id': '3aaf016JFA2oWs6', 'function': {'arguments': '{"query": "2024年意大利队尤文图斯球员"}', 'name': '2024年欧洲杯各球队球衣号码汇总'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 342, 'total_tokens': 408}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ff0956eb-8f0d-4959-b8dc-7dda3c8a3f1a-0', tool_calls=[{'name': '2024年欧洲杯各球队球衣号码汇总', 'args': {'query': '2024年意大利队尤文图斯球员'}, 'id': '3aaf016JFA2oWs6'}], usage_metadata={'input_tokens': 342, 'output_tokens': 66, 'total_tokens': 408})]}}
----
{'tools': {'messages': [ToolMessage(content='林联合）23号：伊武希奇（帕福斯）24号：马尔科·帕萨利奇（里耶卡）25号：苏契奇（萨尔茨堡红牛）26号：巴图里纳（萨格勒布迪纳摩）2024年欧洲杯意大利球员号码：1号：多纳鲁马（巴黎圣日耳曼）2号：迪洛伦佐（那不勒斯）3号：迪马尔科（国际米兰）4号：布翁乔尔诺（都灵）5号：卡拉菲奥里（博洛尼亚）6号：加蒂（尤文图斯）7号：弗拉泰西（国际米兰）8号：若日尼奥（阿森纳）9号：斯卡马卡（亚特兰大）10号：佩莱格里尼（罗马）11号：拉斯帕多里（那不勒斯）1

以上，代理没有将我们的查询逐字逐句地插入到工具中，而是删除了“什么”和“是”等不必要的单词。

同样的原则允许代理在必要时使用对话的上下文：

### 将上面步骤结合在一起
为方便起见，我们将所有必要的步骤都整合到一个代码单元中

In [32]:
import bs4
from langchain.agents import AgentExecutor,create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_community.embeddings import BaichuanTextEmbeddings

llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
    temperature=0,
)
embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])
memory = SqliteSaver.from_conn_string(":memory:")


### 构造检查器 ###
loader = WebBaseLoader(
    web_paths=("https://ouzhoubei.co/article-794-1.html",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(id=("article_content",)))
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vectorstore.as_retriever()

### 构建检索工具 ###
tool = create_retriever_tool(
    retriever,
    "2024年欧洲杯各球队球衣号码汇总",
    "搜索并返回2024年欧洲杯各球队球衣号码文章的摘录。"
)
tools=[tool]

agent_executor = create_react_agent(llm,tools,checkpointer=memory)

In [33]:
config = {
    "configurable":{
        "thread_id":"t2"
    }
}
agent_executor.invoke({"messages":[HumanMessage(content="列出2024年意大利队中属于尤文图斯俱乐部的球员信息，按照：号码-姓名-俱乐部的格式输出。")]},config)

{'messages': [HumanMessage(content='列出2024年意大利队中属于尤文图斯俱乐部的球员信息，按照：号码-姓名-俱乐部的格式输出。', id='38725819-beda-42c0-a2e6-e7b7ca2d6a62'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '72bc016JFCSoRAT', 'function': {'arguments': '{"query": "2024年欧洲杯意大利队名单"}', 'name': '2024年欧洲杯各球队球衣号码汇总'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 319, 'total_tokens': 352}, 'model_name': 'Baichuan4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-63740b8a-801b-489a-9241-b08c18acd045-0', tool_calls=[{'name': '2024年欧洲杯各球队球衣号码汇总', 'args': {'query': '2024年欧洲杯意大利队名单'}, 'id': '72bc016JFCSoRAT'}], usage_metadata={'input_tokens': 319, 'output_tokens': 33, 'total_tokens': 352}),
  ToolMessage(content='林联合）23号：伊武希奇（帕福斯）24号：马尔科·帕萨利奇（里耶卡）25号：苏契奇（萨尔茨堡红牛）26号：巴图里纳（萨格勒布迪纳摩）2024年欧洲杯意大利球员号码：1号：多纳鲁马（巴黎圣日耳曼）2号：迪洛伦佐（那不勒斯）3号：迪马尔科（国际米兰）4号：布翁乔尔诺（都灵）5号：卡拉菲奥里（博洛尼亚）6号：加蒂（尤文图斯）7号：弗拉泰西（国际米兰）8号：若日尼奥（阿森纳）9号：斯卡马卡（亚特兰大）10号：佩莱格里尼（罗马